In [1]:
%load_ext google.cloud.bigquery

In [2]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
client = bigquery.Client()

In [4]:
test_query = "SELECT * FROM `my-msc-project-345300.MIMIC_III.patients` limit 50"
results = client.query(test_query)

In [5]:
dataframe = client.query(test_query).result().to_dataframe()

In [6]:
dataframe.info
dataframe.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13,NaT,NaT,NaT,0
1,238,253,F,2089-11-26,NaT,NaT,NaT,0
2,242,258,F,2124-09-19,NaT,NaT,NaT,0
3,243,260,F,2105-03-23,NaT,NaT,NaT,0
4,247,264,F,2162-11-30,NaT,NaT,NaT,0
